In [5]:
import glob
import json
from tqdm import tqdm
import os
import json
import pandas as pd
import re
import polars as pl

# Download and extract Wiki data

Download and extract a dump from Wikipedia
```
wget -O simplewiki-20250301-pages-articles-multistream.xml.bz2 https://dumps.wikimedia.org/simplewiki/20250301/simplewiki-20250301-pages-articles-multistream.xml.bz2
```

```
bzip2 -d simplewiki-20250301-pages-articles-multistream.xml.bz2 
```

Use WikiExtractor to convert XML into a number of JSON files
```
python -m wikiextractor.WikiExtractor simplewiki-20250301-pages-articles-multistream.xml --json --output extracted
```

In [2]:
files = glob.glob('../extracted/*/*')

In [3]:
files

['../extracted/AA/wiki_28',
 '../extracted/AA/wiki_70',
 '../extracted/AA/wiki_72',
 '../extracted/AA/wiki_95',
 '../extracted/AA/wiki_09',
 '../extracted/AA/wiki_81',
 '../extracted/AA/wiki_99',
 '../extracted/AA/wiki_17',
 '../extracted/AA/wiki_03',
 '../extracted/AA/wiki_20',
 '../extracted/AA/wiki_00',
 '../extracted/AA/wiki_05',
 '../extracted/AA/wiki_47',
 '../extracted/AA/wiki_21',
 '../extracted/AA/wiki_42',
 '../extracted/AA/wiki_11',
 '../extracted/AA/wiki_76',
 '../extracted/AA/wiki_57',
 '../extracted/AA/wiki_51',
 '../extracted/AA/wiki_02',
 '../extracted/AA/wiki_88',
 '../extracted/AA/wiki_53',
 '../extracted/AA/wiki_46',
 '../extracted/AA/wiki_39',
 '../extracted/AA/wiki_49',
 '../extracted/AA/wiki_12',
 '../extracted/AA/wiki_61',
 '../extracted/AA/wiki_59',
 '../extracted/AA/wiki_29',
 '../extracted/AA/wiki_16',
 '../extracted/AA/wiki_14',
 '../extracted/AA/wiki_54',
 '../extracted/AA/wiki_67',
 '../extracted/AA/wiki_50',
 '../extracted/AA/wiki_66',
 '../extracted/AA/wi

In [6]:
data_rows = []

for file_path in tqdm(files):
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
            try:
                record = json.loads(line)
                title = record.get("title", "")
                text = record.get("text", "")
                                
                data_rows.append({
                    "filename": file_path,
                    "title": title,
                    "text": text,
                    "number_of_characters": len(text),
                    "number_of_words": len(text.split()),
                    "topic": 'N-A',
                    "text_quality": 0,
                    
                })
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON in file {file_path}: {e}")

df = pl.DataFrame(data_rows)
print(df)

100%|█████████████████████████████████████████████████████| 233/233 [00:03<00:00, 68.26it/s]


shape: (367_509, 7)
┌───────────────┬──────────────┬──────────────┬──────────────┬──────────────┬───────┬──────────────┐
│ filename      ┆ title        ┆ text         ┆ number_of_ch ┆ number_of_wo ┆ topic ┆ text_quality │
│ ---           ┆ ---          ┆ ---          ┆ aracters     ┆ rds          ┆ ---   ┆ ---          │
│ str           ┆ str          ┆ str          ┆ ---          ┆ ---          ┆ str   ┆ i64          │
│               ┆              ┆              ┆ i64          ┆ i64          ┆       ┆              │
╞═══════════════╪══════════════╪══════════════╪══════════════╪══════════════╪═══════╪══════════════╡
│ ../extracted/ ┆ Card Sharks  ┆ Card Sharks  ┆ 2818         ┆ 514          ┆ N-A   ┆ 0            │
│ AA/wiki_28    ┆              ┆ is a game    ┆              ┆              ┆       ┆              │
│               ┆              ┆ show tha…    ┆              ┆              ┆       ┆              │
│ ../extracted/ ┆ Family Feud  ┆ Family Feud  ┆ 3347         ┆ 611     

In [9]:
df

filename,title,text,number_of_characters,number_of_words,topic,text_quality
str,str,str,i64,i64,str,i64
"""../extracted/AA/wiki_28""","""Card Sharks""","""Card Sharks is a game show tha…",2818,514,"""N-A""",0
"""../extracted/AA/wiki_28""","""Family Feud""","""Family Feud is an American tel…",3347,611,"""N-A""",0
"""../extracted/AA/wiki_28""","""Bank of America Stadium""","""The Bank of America Stadium is…",226,40,"""N-A""",0
"""../extracted/AA/wiki_28""","""Aston Martin""","""Aston Martin Lagonda Limited i…",1064,192,"""N-A""",0
"""../extracted/AA/wiki_28""","""Sydney Opera House""","""The Adil's Sydney Opera House …",1143,198,"""N-A""",0
…,…,…,…,…,…,…
"""../extracted/AC/wiki_13""","""Dirk VI of Holland""","""Dirk VI (c. 1114 – 5 August 11…",250,48,"""N-A""",0
"""../extracted/AC/wiki_13""","""Floris III of Holland""","""Floris III (1141 – August 1, 1…",147,30,"""N-A""",0
"""../extracted/AC/wiki_13""","""Newcastle Breakers FC""","""Newcastle Breakers Football Cl…",129,20,"""N-A""",0


In [8]:
df.write_parquet('simple_english.parquet')

In [10]:
# FineWeb Edu

import polars as pl
x= pl.scan_parquet('./000_00000.parquet')
x.head().collect()

text,id,dump,url,date,file_path,language,language_score,token_count,score,int_score
str,str,str,str,str,str,str,f64,i64,f64,i64
"""This project is solving the As…","""<urn:uuid:6c17be7f-7253-4703-a…","""CC-MAIN-2024-51""","""http://2014.spaceappschallenge…","""2024-12-01T17:43:40Z""","""s3://commoncrawl/crawl-data/CC…","""en""",0.914297,825,2.96875,3
"""Life of a Sand Grain THE LIFE …","""<urn:uuid:17e4ca7f-a10a-461b-8…","""CC-MAIN-2024-51""","""http://blog.azgs.arizona.edu/i…","""2024-12-01T18:24:12Z""","""s3://commoncrawl/crawl-data/CC…","""en""",0.959785,2344,3.546875,4
"""The internet of things, a syst…","""<urn:uuid:6e9f6c85-2d16-4f82-b…","""CC-MAIN-2024-51""","""http://codetechsolutions.co.uk…","""2024-12-01T17:12:59Z""","""s3://commoncrawl/crawl-data/CC…","""en""",0.931961,270,3.484375,3
"""An archive photo of an Egyptia…","""<urn:uuid:47064b08-fcce-4923-9…","""CC-MAIN-2024-51""","""http://en.aswatmasriya.com/new…","""2024-12-01T17:42:22Z""","""s3://commoncrawl/crawl-data/CC…","""en""",0.961632,367,2.75,3
"""The faith of the Christ-God is…","""<urn:uuid:e9c8abb4-8d22-4031-a…","""CC-MAIN-2024-51""","""http://faithandheritage.com/20…","""2024-12-01T18:21:21Z""","""s3://commoncrawl/crawl-data/CC…","""en""",0.958506,2450,2.6875,3


In [11]:
x= pl.scan_parquet('./000_00000.parquet')
x = x.filter((pl.col('token_count')>100)&(pl.col('score')>2.5)&(pl.col('language')=='en'))
x = x.collect()

In [12]:
print(x.filter(x['language_score']<0.7)['text'][0])

In the translated text above, these terms all get the articles, however, in the test, they all don't.
i.e.:la astronomía, y en las matemáticas. También, trabajaron en la agricultura y el comercio"
in the test, it corrected me, so I'm a bit confused:
Could you please let me know when you can/cannot use the article "le,la,las" etc in front of these:
The test examples:
Kwizbot Los aztecas destacaron en astronomía,
You Los aztecas destacan en la astronomía,
You could also say: Los aztecas destacaron en la astronomía,
Kwizbot También, trabajaron en la agricultura
You También, trabajaron en agricultura
[note: here I dropped the article and it didn't correct me]
Well done! and in mathematics.
Your answer matched mine: y en matemáticas.
Thank you for your help and have a great day!
Freeform Writing Exercise A2
Four years later, and still no answer to this question. I had the same questions. It appears to me using various online resources, grammar correcting sites, and ChatGPT that the use of a

In [ ]:
print(x.filter(x['score']<2.7)['text'][0])

In [17]:
x['token_count'].sum()

412145408

# Let's build our own data classifier!

In [14]:
import pandas as pd
import google.generativeai as genai
import os
import time
import json
from dotenv import load_dotenv
from tqdm import tqdm  # Import tqdm for progress bar

load_dotenv()
API_KEY = os.getenv("MY_API_KEY")

if not API_KEY:
    raise ValueError("Google API Key not found. Please set the GOOGLE_API_KEY environment variable or define it in the script.")

genai.configure(api_key=API_KEY)


MODEL_NAME = "gemini-1.5-flash-8b"

ALLOWED_TOPICS = ["Art", "History", "Science", "General knowledge"]
ALLOWED_COMPLEXITY = ["elementary", "middle school", "high school", "university degree", "doctorate"]

def classify_text_with_gemini(text_to_classify):
    """
    Uses Gemini to classify text into topic and complexity.

    Args:
        text_to_classify (str): The text content to classify.

    Returns:
        tuple: (topic, complexity) or (None, None) if an error occurs.
    """
    # Ensure text is not empty or just whitespace
    if not text_to_classify or text_to_classify.isspace():
        print("Warning: Skipping empty or whitespace-only text.")
        return None, None

    # Limit text length to avoid exceeding model limits (adjust as needed)
    max_length = 15000 # Example limit, check model's token limits if necessary
    if len(text_to_classify) > max_length:
        text_to_classify = text_to_classify[:max_length] + "..." # Truncate


    prompt = f"""
Analyze the following text and determine its primary topic and the educational level typically required to understand it.

Text:
"{text_to_classify}"

Instructions:
1. Choose the *single best* topic from this list: {ALLOWED_TOPICS}
2. Choose the *single most appropriate* complexity level from this list: {ALLOWED_COMPLEXITY}
3. Provide your answer ONLY in the following JSON format:
   {{"topic": "SELECTED_TOPIC", "complexity": "SELECTED_COMPLEXITY"}}

Example Response:
{{"topic": "Science", "complexity": "high school"}}

Output ONLY the JSON object.
"""

    try:
        model = genai.GenerativeModel('gemini-1.5-flash-8b')
        response = model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                # candidate_count=1, # Ensure only one response candidate
                # max_output_tokens=50, # Limit output size
                temperature=0.1 # Lower temperature for more deterministic classification
            )
            )

        raw_response_text = response.text.strip()
        if raw_response_text.startswith("```json"):
            raw_response_text = raw_response_text[7:] # Remove ```json
        if raw_response_text.endswith("```"):
            raw_response_text = raw_response_text[:-3] # Remove ```
        raw_response_text = raw_response_text.strip() # Clean whitespace again

        # Parse the JSON response
        result = json.loads(raw_response_text)
        topic = result.get("topic")
        complexity = result.get("complexity")

        # Validate response against allowed categories
        if topic not in ALLOWED_TOPICS:
            print(f"Warning: Received invalid topic '{topic}'. Setting to None.")
            topic = None
        if complexity not in ALLOWED_COMPLEXITY:
            print(f"Warning: Received invalid complexity '{complexity}'. Setting to None.")
            complexity = None

        return topic, complexity

    except json.JSONDecodeError:
        print(f"Error: Failed to decode JSON response: {raw_response_text}")
        return "Error: JSON Decode", "Error: JSON Decode"
    except ValueError as ve: # Handle potential errors from model generation (e.g., blocked content)
         print(f"Error: Gemini API returned ValueError (potentially blocked content or invalid response structure). Raw response: {response.text if 'response' in locals() else 'N/A'}. Details: {ve}")
         return "Error: API Value", "Error: API Value"
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return "Error: API Call", "Error: API Call"

/home/luca/jupyterlab310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
classify_text_with_gemini("LLMs in simple, pure C/CUDA with no need for 245MB of PyTorch or 107MB of cPython. ")

('Science', 'university degree')

In [ ]:
import concurrent.futures
MAX_WORKERS = 8
results = [None] * len(df) # Pre-allocate list to store results in order

with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:

    future_to_index = {
        executor.submit(classify_text_with_gemini, text): i
        for i, text in enumerate(df['text'])
    }

    for future in tqdm(concurrent.futures.as_completed(future_to_index), total=len(df), desc="Classifying Text"):
        index = future_to_index[future]
        try:
            result_tuple = future.result()
            results[index] = result_tuple
        except Exception as exc:
            print(f'Row {index} generated an exception: {exc}')
            results[index] = ("Error: Future Exception", "Error: Future Exception")

Classifying Text:   0%|                               | 1/367509 [00:00<53:49:13,  1.90it/s]

Classifying Text:   0%|                               | 121/367509 [00:00<32:01, 191.24it/s]

Classifying Text:   0%|                               | 149/367509 [00:01<54:41, 111.95it/s]

Classifying Text:   0%|                              | 166/367509 [00:01<1:12:29, 84.45it/s]

Classifying Text:   0%|                              | 178/367509 [00:02<1:27:11, 70.22it/s]

...

Classifying Text:   8%|██▏                         | 29306/367509 [12:29<2:06:51, 44.44it/s]

Classifying Text:   8%|██▏                         | 29319/367509 [12:29<1:59:22, 47.22it/s]

Classifying Text:   8%|██▏                         | 29330/367509 [12:30<2:22:58, 39.42it/s]

Classifying Text:   8%|██▏                         | 29340/367509 [12:30<2:32:59, 36.84it/s]

Classifying Text:   8%|██▏                         | 29345/367509 [12:30<2:36:35, 35.99it/s]

In [ ]:
# Generating synthetic data

In [19]:
import json

def generate_synthetic_data(sample_text):
    """
    Uses Gemini to generate synthetic data from a sample text. The synthetic data is obtained by summarizing,
    paraphrasing, extracting keywords, and creating a creative variant of the text.

    Args:
        sample_text (str): The sample text on which to base the synthetic data generation.

    Returns:
        dict: A dictionary containing synthesized data with the following keys:
              - "summary": A succinct summary of the text.
              - "paraphrase": A reworded version of the text.
              - "keywords": A list of keywords extracted from the text.
              - "synthetic_variant": A creatively modified variant of the text.
              Returns error messages in the dictionary fields if an error occurs.
    """
    # Ensure text is not empty or just whitespace
    if not sample_text or sample_text.isspace():
        print("Warning: Skipping empty or whitespace-only text.")
        return {
            "summary": None,
            "paraphrase": None,
            "keywords": None,
            "synthetic_variant": None,
        }

    # Limit text length to avoid exceeding model limits (adjust as needed)
    max_length = 15000  # Example limit, adjust based on model token limits if needed
    if len(sample_text) > max_length:
        sample_text = sample_text[:max_length] + "..."  # Truncate

    prompt = f"""
You are an advanced language model designed to perform text augmentation. Given the following text, your task is to generate synthetic data by performing these four tasks:

1. **Summarization**: Provide a succinct summary of the text.
2. **Paraphrasing**: Rephrase the text while preserving its meaning.
3. **Synthetic Variant**: Generate one additional variant of the text with creative modifications (e.g. varying sentence structure or introducing subtle stylistic changes).

Please provide your answer ONLY in the following JSON format:
{{
    "summary": "Summarized version of the text.",
    "paraphrase": "Paraphrased version of the text.",
    "synthetic_variant": "Creative synthetic variant of the text."
}}

Example Response:
{{
    "summary": "A brief summary of the sample text.",
    "paraphrase": "A reworded version of the sample text.",
    "synthetic_variant": "An alternative version of the text with creative modifications."
}}

Text:
\"{sample_text}\"

Output ONLY the JSON object.
"""

    try:
        model = genai.GenerativeModel("gemini-1.5-flash")  # Replace with your actual model name
        response = model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.2,  # Adjust temperature to balance creativity and fidelity
                # You can also fine-tune the max output tokens if necessary, e.g.
                # max_output_tokens=150,
            )
        )

        raw_response_text = response.text.strip()
        # Remove markdown formatting if present
        if raw_response_text.startswith("```json"):
            raw_response_text = raw_response_text[7:]
        if raw_response_text.endswith("```"):
            raw_response_text = raw_response_text[:-3]
        raw_response_text = raw_response_text.strip()

        # Parse the JSON response
        result = json.loads(raw_response_text)
        # Optionally, validate that required keys are present
        for key in ["summary", "paraphrase", "synthetic_variant"]:
            if key not in result:
                print(f"Warning: Expected key '{key}' not found in the output. Setting its value to None.")
                result[key] = None

        return result

    except json.JSONDecodeError:
        print(f"Error: Failed to decode JSON response: {raw_response_text}")
        return {
            "summary": "Error: JSON Decode",
            "paraphrase": "Error: JSON Decode",
            "synthetic_variant": "Error: JSON Decode",
        }
    except ValueError as ve:
        print(f"Error: Gemini API returned ValueError. Raw response: {response.text if 'response' in locals() else 'N/A'}. Details: {ve}")
        return {
            "summary": "Error: API Value",
            "paraphrase": "Error: API Value",
            "synthetic_variant": "Error: API Value",
        }
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return {
            "summary": "Error: API Call",
            "paraphrase": "Error: API Call",
            "synthetic_variant": "Error: API Call",
        }

In [20]:
generate_synthetic_data(df['text'][0])

{'summary': 'Card Sharks, a game show with various iterations since 1978, features contestants guessing card values higher or lower than previous ones to win money.  Different versions existed, with varying rules and hosts, including Jim Perry, Bob Eubanks, and Joel McHale. A 2001 version with teams and different rules proved unpopular.',
 'paraphrase': 'Since its debut in 1978, the game show Card Sharks has undergone several revivals.  The core gameplay involves contestants competing to predict whether playing cards will be higher or lower in value than the preceding card, accumulating money with successful guesses.  Different hosts and rule variations have marked its history, with some versions proving more successful than others.',
 'synthetic_variant': "From its 1978 premiere under Jim Perry's charismatic hosting, Card Sharks has captivated audiences with its simple yet thrilling premise: outguess your opponent on a sequence of cards, higher or lower, to win cash.  Through various 

# Part 2

# Loading the tokenizer

In [21]:
from transformers import PreTrainedTokenizerFast

enc = PreTrainedTokenizerFast.from_pretrained('./dev/data/simple_english.parquet/custom_tokenizer')

In [4]:
import numpy as np

In [5]:
with open('./dev/data/simple_english.parquet/simple_english.parquet_custom_train_000041.bin', "rb") as f:
    # first read the header, which is 256 int32 integers (4 bytes each)
    header = np.frombuffer(f.read(256*4), dtype=np.int32)
    ntok = header[2] # number of tokens (claimed)
    # the rest of it are tokens, stored as uint16
    tokens = np.frombuffer(f.read(), dtype=np.uint32)

In [6]:
decoded_text = enc.decode(tokens[100:200]).replace(" ", "").replace("Ġ", " ").replace(" ##", "").replace('Ċ','\n')

In [7]:
print(decoded_text)

 originated from English speaking countries. It finds seldom use in other metricated Commonwealth Nations.</s></s></s> Shohreh Solati (, born Fatemeh Solati on January 4, 1959, in Tehran) is an Iranian singer. She is among the most active and prolific Iranian female singers. Since the Iranian Revolution, she has continued her music career outside the country.
She is divorced with a daughter and lives in Los Angeles, California.</s></s>


In [11]:
import polars as pl
pl.read_parquet('simple_english.parquet')['number_of_words'].sum()

32077573